<a href="https://colab.research.google.com/github/MarvelManueke/Machine-Learning-Projects/blob/main/book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-08-07 14:59:37--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   104MB/s    in 0.2s    

2023-08-07 14:59:37 (104 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Count the number of ratings for each user and isbn
user_rating_counts = df_ratings['user'].value_counts()
isbn_counts = df_ratings['isbn'].value_counts()

# Get the user IDs with less than 200 ratings and books with less than 100 ratings
users_to_drop = user_rating_counts[user_rating_counts < 200].index
isbn_drop = isbn_counts[isbn_counts < 100].index

# Drop data for users with less than 200 ratings and books
df_merge = df_ratings[~df_ratings['user'].isin(users_to_drop)]
df_merge = df_merge[~df_merge['isbn'].isin(isbn_drop)]

book_rating = pd.merge(df_merge, df_books, on = "isbn")
book_rating = book_rating.drop_duplicates(["title", "user"])
book_rating.shape

(49136, 5)

In [ ]:
br_pivot = book_rating.pivot_table(index='title', columns='user',values='rating').fillna(0)
br_pivot_matrix = csr_matrix(br_pivot.values)

In [ ]:
model_nn = NearestNeighbors(n_neighbors = 6,
                            metric="cosine",
                           p=2)
model_nn.fit(br_pivot_matrix)

NearestNeighbors(metric='cosine', n_neighbors=6)

In [ ]:
def get_recommends(book=""):
    recommended_books = model_nn.kneighbors(br_pivot.loc[[book]], 6)  # Request one additional neighbor
    recommended_indices = recommended_books[1][0]
    recommended_distances = recommended_books[0][0]

    recommended_titles = [br_pivot.index[i] for i in recommended_indices]
    input_title_index = recommended_titles.index(book)
    recommended_titles.pop(input_title_index)  # Remove the input book title from recommendations

    recommendations_with_distances = list(zip(recommended_titles, recommended_distances))

    # Reverse the order of recommendations based on distances
    recommendations_with_distances.reverse()

    output = [book, [list(item) for item in recommendations_with_distances]]
    return output

# Example usage
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [["I'll Be Seeing You", 0.77085835],
  ['The Weight of Water', 0.7699411],
  ['The Surgeon', 0.7677075],
  ['I Know This Much Is True', 0.7234864],
  ['The Lovely Bones: A Novel', 0.0]]]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.77085835], ['The Weight of Water', 0.7699411], ['The Surgeon', 0.7677075], ['I Know This Much Is True', 0.7234864], ['The Lovely Bones: A Novel', 0.0]]]
You passed the challenge! 🎉🎉🎉🎉🎉
